# EE 304 - Neuromorphics: Brains in Silicon


##  Approximating Functions

#### The story thus far

We considered how to implement synaptic weights in architectures that use shared synapses (i.e. Shared Synapse and Shared Dendrite). 

We found that a probabilistic approach is compatible with shared synapses. 

We derived expressions for the mean and variance of the total synaptic input a neuron receives in the deterministic and  probabilistic cases. 

We compared the memory look-ups and spike-traffic these two approaches perform and generate, respectively, to achieve a specified the signal-to-noise ratio ($r$) at the input of a neuron.

We found that, for the probabilistic approach to be competitive, $r$ must be significantly less than the square-root of the number of neurons ($N$). 

This leads to the question: How big must $N$ be?

####  Outline for this lecture

<img src="files/lecture14/NEF+ClassicNets.pdf" width="840">

We will study how the approximation to a function ($f(x)$) obtained by taking a weighted sum of tuning curves ($a_i(x)$) depends on the number of neurons $N$. 

...

###  NEF Decoders: Problem Statement 

<img src="files/lecture14/TuningCurves+f=Ad.pdf" width="960">

In NEF, the decoders are obtained by solving the following problem: 

- Express a function $f(x)$ as a weighted sum of $N$ neural tuning curves $a_i(x)$.
- The “decoding” weights are labeled $d_i$. 
- The function and tuning curves are sampled at $Q$ points, yielding a $Q\times 1$ vector ${\bf f}$ and a $Q \times N$ matrix ${\bf A}$, respectively. 
- The $N$ decoding weights, packed in a $N\times 1$ vector ${\bf d}$, are then obtained by solving the matrix equation ${\bf f} = {\bf Ad}$.


###  Singular Value Decomposition 

$$
        {\bf A} = 
            \left[\begin{array}{c|c|c|c} {\bf u}_1 & {\bf u}_2 & \cdots & {\bf u}_Q \end{array}\right]
            \left[\begin{array}{} 
                s_1    & 0      & \cdots & 0      \\
                0      & s_2    & \cdots & 0      \\
                \vdots & \vdots & \ddots & \vdots \\
                0      & 0      & \cdots & s_N    \\
                0      & 0      & \cdots & 0      \\
                \vdots & \vdots & \cdots & \vdots
            \end{array}\right]
            \left[\begin{array}{} {\bf v}_1 \\ \hline  {\bf v}_2 \\ \hline  \vdots \\ {\bf v}_N \end{array}\right]
$$

${\bf A}$’s singular value decomposition (SVD) is:
$$
    {\bf A} = {\bf U}{\bf S}{\bf V}^{\rm T} = \sum_m^N s_m{\bf u}_m{\bf v}_m^{\rm T}
$$

- ${\bf U}$ is a $Q\times Q$ matrix 
    - It’s columns, ${\bf u}_m$, are an orthonormal basis for the $Q$-dimensional space
    - ${\bf U}^{\rm T}{\bf U} = {\bf U}{\bf U}^{\rm T} = {\bf I}$
- ${\bf S}$ is a $Q \times N$ diagonal matrix 
    - It’s entries, ${\bf s}_m$, are ${\bf A}$’s singular values.
- ${\bf V}$ is an $N\times N$ matrix 
    - It’s columns, ${\bf v}_m$, are an orthonormal basis for the $N$-dimensional space
    - ${\bf V}^{\rm T}{\bf V} = {\bf V}{\bf V}^{\rm T} = {\bf I}$

### Solving for Decoders using SVD 

$$
        {\bf d} = 
            \left[\begin{array}{c|c} {\bf v}_1 & {\bf v}_2 & \cdots & {\bf v}_N \end{array}\right]
            \left[\begin{array}{} 
                s_1^{-1} & 0        & \cdots & 0        & 0      & \cdots \\
                0        & s_2^{-1} & \cdots & 0        & 0      & \cdots \\
                \vdots   & \vdots   & \ddots & \vdots   & \vdots & \vdots \\
                0        & 0        & \cdots & s_N^{-1} & 0      & \cdots
            \end{array}\right]
            \left[\begin{array}{} c_1 \\ c_2 \\ \vdots \\ c_Q \end{array}\right]
$$


Using SVD, we can solve for ${\bf d}$ as follows:
$$
    {\bf f} = {\bf U}{\bf S}{\bf V}^{\rm T}{\bf d} \implies {\bf d} = {\bf V}{\bf S}^{-1}{\bf U}^{\rm T}{\bf f}
$$

Writing ${\bf f}$'s projection onto ${\bf U}$ as 
$$
    {\bf c} = {\bf U}^{\rm T}{\bf f} \;{\rm or}\; c_q = {\bf u}_q^{\rm T}{\bf f} 
$$

And substituting into the expression for ${\bf d}$ above yields
$$
    {\bf d} = {\bf V}{\bf S}^{-1}{\bf c} = \sum_m^N \frac{c_m}{s_m}{\bf v}_m
$$

This solution is equivalent to the least-squared-error solution.

### Decoder's Error without Noise 

The root-mean-square error (RMSE) is:

\begin{eqnarray*}
    E & = & \frac{1}{\surd Q}\| {\bf f}-{\bf Ad}\|\\
      & = & \frac{1}{\surd Q}\left\| 
            \sum_q^Q c_q{\bf u}_q 
            -\left(\sum_q^N s_q{\bf u}_q{\bf v}_q^{\rm T}\right)
             \left(\sum_m^N \frac{c_m}{s_m}{\bf v}_m\right)\right\|\\
      & = & \frac{1}{\surd Q}\left\|\sum_q^Q c_q{\bf u}_q - \sum_q^N c_q{\bf u}_q\right\|\\
      & = & \frac{1}{\surd Q}\left\|\sum_{q=N+1}^Q c_q{\bf u}_q\right\|\\
      & = & \sqrt{\frac{1}{Q}\sum_{q=N+1}^Q c_q^2}
\end{eqnarray*}

Thus, error arises because $N < Q$:

- $N$-dimensional neural patterns cannot fully represent a $Q$-dimensional function.
- This prediction is quantitatively accurate ({meas, pred} above).


### Effect of Noise on Singular Values and Vectors

<img src="files/lecture14/NoisySingularValues+Vectors.pdf" width="960">

Benaych-Georges & Nadakuditi (2012) and Gavish & Donoho (2014) derive the asymptotic behavior of random matrixes:

- As the number of rows tends to infinity (i.e., $N \to \infty$)
- While keeping the ratio between numbers of rows and columns (i.e, $N/Q$) constant 

Their theory may be used to relate the noisy measurment matrix's ($\tilde{\bf A}$) singular values and vectors to those of the noiseless matrix (${\bf A}$).

It yields expressions that relate:

- The new singular values to the old ones (left panel)
    - Noise boosts small values, shrinking the overall range
- The inner product between a new and an old singular vector to the old singular value (middle and right panels)
    - It becomes zero when the old singular-value drops below a cut-off 
    - The red line in the left panel indicates this cut-off

The examples shown, and those in the rest of this presentation, use square-root tuning curves (see Slide 3) with N = 200, Q = 400, and σ = 0.1. 

### Asymptotic Analytical Results 

<img src="files/lecture14/SingularValueNoiselessToNoisy.pdf" width="480">

In the case where normally-distributed noise (with variance $\sigma^2$ and mean zero) is added to the entries of ${\bf A}$ to obtain $\tilde{\bf A}$:

- $\tilde{\bf A}$'s singular values are related to ${\bf A}$'s by

$$
    \tilde{s}_q = s_q\sqrt{\left(1+Q\sigma^2/s_q^2\right)\left(1+N\sigma^2/s_q^2\right)}
$$

- And the inner products of their singular vectors are

$$
    {\bf u}^{\rm T}{\bf u}=\sqrt{\frac{1-NQ\sigma^4/s_q^4}{1+Q\sigma^2/s_q^2}}
    \;\;{\rm and}\;\;
    {\bf v}^{\rm T}{\bf v}=\sqrt{\frac{1-NQ\sigma^4/s_q^4}{1+N\sigma^2/s_q^2}}
$$

- if $s_q^2/\sigma^2>\sqrt{NQ}$. Otherwise, the inner product is zero.


### Decoder's Error with Noise

The RMSE on a test set $\hat{A}=\hat{\bf U}\hat{\bf S}\hat{\bf V}^{\rm T}$ is:
\begin{eqnarray*}
    E & = & \frac{1}{\surd Q}\| {\bf f}-{\bf Ad}\|\\
      & = & \frac{1}{\surd Q}\left\| 
            \sum_q^Q c_q{\bf u}_q 
            -\left(\sum_q^N \hat{s}_q\hat{\bf u}_q\hat{\bf v}_q^{\rm T}\right)
             \left(\sum_m^N \frac{c_m}{s_m}{\bf v}_m\right)\right\|\\
      & = & \frac{1}{\surd Q}\left\|\sum_q^Q c_q{\bf u}_q 
            - \sum_q^r \frac{\hat{s}_q}{s_q}c_q\hat{\bf u}_q\hat{\bf v}_q^{\rm T}{\bf v}_q\right\|\\
      & = & \frac{1}{\surd Q}\left\|
              \sum_{q=N+1}^Q c_q\left({\bf u}_q-\frac{\hat{s}_q}{s_q}\hat{\bf v}_q^{\rm T}{\bf v}_q\hat{\bf u}_q\right)
              + \sum_{q=r+1}^Q c_q{\bf u}_q\right\|\\
      & = & \frac{1}{\surd Q}
            \sqrt{
              \sum_q^r \left(
                  1 + \left(\frac{\hat{s}_q}{s_q}\hat{\bf v}_q^{\rm T}{\bf v}_q\right)^2 
                  - 2\frac{\hat{s}_q}{s_q}\hat{\bf v}_q^{\rm T}{\bf v}_q\hat{\bf u}_q^{\rm T}{\bf u}_q
                  \right)c_q^2
              + \sum_{q=r+1}^Q c_q^2
              }
\end{eqnarray*}
